<a href="https://colab.research.google.com/github/jumanajouhar/traffic_sign_recognition/blob/main/yolov8_sahi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Change to the correct directory in your Google Drive
import os
from pathlib import Path

# Change to the directory containing your video and model weight files
os.chdir("/content/drive/My Drive/traffic_sign_recognition")

# Verify the current working directory
print("Current working directory:", os.getcwd())

# List the files to ensure 'video.mp4' and 'road.pt' are in the folder
!ls

# Check if the required files are in the directory
if not Path("video5.mp4").exists():
    raise FileNotFoundError("video not found in the current directory.")
if not Path("road.pt").exists():
    raise FileNotFoundError("road.pt not found in the current directory.")

# Step 3: Install necessary libraries
!pip install opencv-python sahi ultralytics

# Step 4: Import libraries
import cv2
from ultralytics import YOLO  # Use YOLO from ultralytics
from ultralytics.utils.files import increment_path

# Step 5: Define the object detection function
def run(weights="road.pt", source="video5.mp4", view_img=False, save_img=True, exist_ok=False, device='cpu'):
    """Run object detection on a video using YOLOv8."""

    CONFIDENCE_THRESHOLD = 0.1  # Lower confidence threshold to capture low-confidence detections

    # Check if the source file exists
    if not Path(source).exists():
        raise FileNotFoundError(f"Source path '{source}' does not exist.")

    # Initialize YOLOv8 model using Ultralytics
    detection_model = YOLO(weights)

    # Ensure the model has class names available
    if not hasattr(detection_model, 'names') or not detection_model.names:
        raise ValueError("Model class names are not available. Please ensure the model is loaded correctly.")

    # Video setup
    videocapture = cv2.VideoCapture(source)
    if not videocapture.isOpened():
        raise Exception(f"Failed to open video file: {source}")

    frame_width, frame_height = int(videocapture.get(3)), int(videocapture.get(4))
    fps, fourcc = int(videocapture.get(5)), cv2.VideoWriter_fourcc(*"mp4v")

    # Output directory setup
    save_dir = increment_path(Path("ultralytics_results_with_sahi") / "exp", exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)
    video_writer = cv2.VideoWriter(str(save_dir / f"{Path(source).stem}.mp4"), fourcc, fps, (frame_width, frame_height))
    print("Results saved to ", save_dir)

    # Loop through the video frame by frame
    while videocapture.isOpened():
        success, frame = videocapture.read()
        if not success:
            break

        # Inference using the YOLOv8 model on each frame
        results = detection_model(frame)

        # Process predictions
        for result in results:  # Loop over the predictions
            for box in result.boxes:  # Accessing the bounding boxes
                bbox = box.xyxy[0]  # Get bounding box coordinates (x1, y1, x2, y2)
                cls = int(box.cls[0])  # Get class index
                score = box.conf[0]  # Get confidence score

                # Only draw the detection if the confidence score is above the threshold
                if score >= CONFIDENCE_THRESHOLD:
                    # Get the class name from the model
                    class_name = detection_model.names[cls] if cls < len(detection_model.names) else "Unknown"

                    # Draw bounding box on the frame
                    cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (56, 56, 255), 2)

                    # Add label OUTSIDE the bounding box (above the box)
                    label = f"{class_name} {score:.2f}"
                    t_size = cv2.getTextSize(label, 0, fontScale=0.6, thickness=1)[0]
                    cv2.rectangle(
                        frame, (int(bbox[0]), int(bbox[1]) - t_size[1] - 10), (int(bbox[0]) + t_size[0], int(bbox[1]) - 10), (56, 56, 255), -1
                    )
                    cv2.putText(
                        frame, label, (int(bbox[0]), int(bbox[1]) - t_size[1] - 3), 0, 0.6, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA
                    )

        # Save the processed frame to the video file
        if save_img:
            video_writer.write(frame)

    # Release video writer and capture objects
    video_writer.release()
    videocapture.release()

# Step 6: Define argument parsing function
def parse_opt():
    """Parse command line arguments for configuration."""
    return {
        "weights": "road.pt",  # Your model weight file
        "source": "video5.mp4",  # Your video file
        "view_img": False,  # Set to True if you want to display the video in the notebook
        "save_img": True,  # Set to True to save the output video
        "exist_ok": False,  # Set to True if you want to overwrite existing results
        "device": 'cpu',  # Use 'cpu' or 'cuda:0' for GPU
    }

# Step 7: Run the main function using the parsed options
opt = parse_opt()
run(**opt)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/My Drive/traffic_sign_recognition
road.pt			       video2.mp4  video4.mp4  video.mp4
ultralytics_results_with_sahi  video3.mp4  video5.mp4  yolov8_sahi.ipynb
Results saved to  ultralytics_results_with_sahi/exp5

0: 384x640 (no detections), 358.4ms
Speed: 4.6ms preprocess, 358.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 189.1ms
Speed: 5.9ms preprocess, 189.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 166.8ms
Speed: 4.5ms preprocess, 166.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 154.9ms
Speed: 4.1ms preprocess, 154.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 157.4ms
Speed: 4.5ms preprocess, 157.4ms inferenc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')